# 신용카드 사용자 연체 예측

월간 데이콘 신용카드 사용자 연체 예측 AI 경진대회

https://dacon.io/competitions/official/235713/overview/description

## Stage 1. 데이터 불러오기

### 1-1. 데이터 불러오기

In [2]:
# pandas 패키지를 pd라는 별칭으로 불러옵니다.
import pandas as pd

In [3]:
# sample_submission.csv 파일을 submission이라는 변수에 할당합니다.
submission = pd.read_csv("../../data/데이콘_신용카드/sample_submission.csv")

In [4]:
# pandas 패키지를 pd 라는 별칭으로 import 해봅시다!
import pandas as pd

# pandas 내에 있는 read_csv 함수를 사용해 train.csv, test.csv 파일을 각각 train, test라는 이름으로 불러와봅시다!
train = pd.read_csv('../../data/데이콘_신용카드/train.csv')
test = pd.read_csv('../../data/데이콘_신용카드/test.csv')

### 1-2. 간단하게 데이터 살펴보기

submission 데이터의 상위 행 5개를 확인

In [5]:
submission.head()

,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0


train 데이터의 하위 행 5개를 확인

In [6]:
train.tail()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
26452,26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NaN,2.0,-47.0,2.0
26454,26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0
26456,26456,F,N,N,0,81000.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-19569,-1013,1,0,0,0,Security staff,2.0,-9.0,2.0


데이터프레임의 columns 속성은 열 이름들을 반환하여 변수명을 확인

In [7]:
train.columns

Index(['index', 'gender', 'car', 'reality', 'child_num', 'income_total',
       'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email',
       'occyp_type', 'family_size', 'begin_month', 'credit'],
      dtype='object')

train 데이터의 컬럼은 다음과 같습니다.

- index  
- gender: 성별  
- car: 차량 소유 여부  
- reality: 부동산 소유 여부  
- child_num: 자녀 수  
- income_total: 연간 소득  
- income_type: 소득 분류
- edu_type: 교육 수준  
- family_type: 결혼 여부  
- house_type: 생활 방식  
- DAYS_BIRTH: 출생일  
- DAYS_EMPLOYED: 업무 시작일  
- FLAG_MOBIL: 핸드폰 소유 여부  
- work_phone: 업무용 전화 소유 여부    
- phone: 전화 소유 여부  
- email: 이메일 소유 여부  
- occyp_type: 직업 유형  
- family_size: 가족 규모  
- begin_month: 신용카드 발급 월  
- credit: 사용자의 신용카드 대금 연체를 기준으로 한 신용도  

데이터프레임의 shape 속성은 데이터프레임의 크기를 반환하는 속성  
shape 속성을 통해 데이터프레임의 행과 열을 확인

In [8]:
train.shape

(26457, 20)

## Stage 2. 데이터 확인

### 2-1. 데이터프레임 정보 확인

info()

결측값 확인 및 데이터타입 확인

verbose=True 옵션 사용시 잘리는 부분 없이 모두 출력

In [9]:
train.info(verbose=True) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

describe()

데이터프레임의 기초 통계량을 나타냄  
크게 튀는 값들에 대해서는 추후 박스플롯 그려내어 이상치값을 제거할수도 있음  

주로 수치형 데이터를 간략하게 파악하고 싶을 때 유용하게 사용

In [10]:
train.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000000,26457.000000,2.645700e+04,26457.000000,26457.000000,26457.0,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000
mean,13228.000000,0.428658,1.873065e+05,-15958.053899,59068.750728,1.0,0.224742,0.294251,0.091280,2.196848,-26.123294,1.519560
std,7637.622372,0.747326,1.018784e+05,4201.589022,137475.427503,0.0,0.417420,0.455714,0.288013,0.916717,16.559550,0.702283
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6614.000000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19842.000000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,19.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,2.000000


DAYS_BIRTH, DAYS_EMPLOYED, begin_month 음수로 값이 나오고 있음

DAYS_BIRTH: 출생일  
-> 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미  

DAYS_EMPLOYED: 업무 시작일  
-> 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을  
시작함을 의미 (양수 값은 고용되지 않은 상태를 의미함)  

begin_month: 신용카드 발급 월  
-> 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미  

DAYS_BIRTH, DAYS_EMPLOYED, begin_month 변수들의 값을 알아보기 쉽게 바꿔줌  

train 데이터의 각 변수들을 날짜 단위가 아닌 연 단위(만 나이)로 바꿔줌   
(begin_month 제외)

In [11]:
# 각 변수들을 날짜 단위가 아닌 연 단위(만 나이)로 바꿔줌 (begin_month 제외)

# train 적용
train.loc[train["DAYS_EMPLOYED"] > 0,"DAYS_EMPLOYED"] = 0
train["DAYS_BIRTH"] = (train["DAYS_BIRTH"] * -1) / 365.0
train["DAYS_EMPLOYED"] = (train["DAYS_EMPLOYED"] * -1) / 365.0
train["begin_month"] = (train["begin_month"] * -1)

# test 적용
test.loc[test["DAYS_EMPLOYED"] > 0,"DAYS_EMPLOYED"] = 0
test["DAYS_BIRTH"] = (test["DAYS_BIRTH"] * -1) / 365.0
test["DAYS_EMPLOYED"] = (test["DAYS_EMPLOYED"] * -1) / 365.0
test["begin_month"] = (test["begin_month"] * -1)

value_counts()

카테고리형 데이터 파악에 유용한 메서드  
변수마다 각 카테고리 값들이 얼마나 있는지 확인  
value_counts(normalize=True)로 파라미터를 설정하면 각 값들의 비율을 확인가능  

In [14]:
# train 데이터의 카테고리 컬럼들을 추출합니다.
categories = train.select_dtypes(exclude='number')

# 추출한 변수들의 값들의 비율을 확인합니다.
for feature in categories:
  print(train[feature].value_counts(normalize=True))
  print('-'*50)

F    0.668897
M    0.331103
Name: gender, dtype: float64
--------------------------------------------------
N    0.620252
Y    0.379748
Name: car, dtype: float64
--------------------------------------------------
Y    0.673924
N    0.326076
Name: reality, dtype: float64
--------------------------------------------------
Working                 0.515743
Commercial associate    0.234418
Pensioner               0.168160
State servant           0.081415
Student                 0.000265
Name: income_type, dtype: float64
--------------------------------------------------
Secondary / secondary special    0.680160
Higher education                 0.270703
Incomplete higher                0.038553
Lower secondary                  0.009714
Academic degree                  0.000869
Name: edu_type, dtype: float64
--------------------------------------------------
Married                 0.687757
Single / not married    0.132139
Civil marriage          0.080243
Separated               0.058170
Wido

### 2-2. 결측치 처리  

info() 메서드로 occyp_type 변수가 결측치가 존재한다는 사실 확인하였음  
occyp_type 변수는 직업을 나타내는 변수이므로 결측치는 '무직'(No job) 생각하고 처리해줌  

In [ ]:
# `occyp_type` 변수의 결측치들을 "No job"으로 채워줌
train["occyp_type"] = train["occyp_type"].fillna("No job")
test["occyp_type"] = test["occyp_type"].fillna("No job")

In [15]:
# occyp_type 변수의 값의 비율을 백분율로 확인합니다.
round(train["occyp_type"].value_counts(normalize=True) * 100, 2)

Laborers                 24.67
Core staff               14.47
Sales staff              13.88
Managers                 11.85
Drivers                   8.61
High skill tech staff     5.69
Accountants               4.93
Medicine staff            4.72
Cooking staff             2.50
Security staff            2.32
Cleaning staff            2.20
Private service staff     1.33
Low-skill Laborers        0.69
Waiters/barmen staff      0.68
Secretaries               0.53
Realty agents             0.34
HR staff                  0.34
IT staff                  0.22
Name: occyp_type, dtype: float64

### 2-3. 전처리 데이터 저장하기

앞에서의 결과를 통해 train 데이터와 test 데이터를 사용하기 편하게  
값을 변경해주었는데 이를 데이터 전처리라고 한다.  

이렇게 전처리된 데이터를 저장하기 위해 pandas의 to_csv() 메서드로 저장해준다.

In [16]:
# # 전처리된 train, test 데이터를 각각 저장

# train.to_csv("creditcard_train.csv", index=False)
# test.to_csv("creditcard_test.csv", index=False)

## Stage 3. 모델링 기초  

.